In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import welch
import numpy as np

FILE_PATH = "../dataset/imu.txt"
# FILE_PATH = "../dataset/RawGyro.csv"

In [2]:
#gyro_accel_data = pd.read_csv('../dataset/RawGyro.csv')
# Read the text file into a DataFrame
gyro_accel_data = pd.read_csv(FILE_PATH,sep=" ")

# Print the type of the DataFrame
gyro_accel_data  # Instead of gyro_accel_data[' timestamp']


,#,timestamp,ang_vel_x,ang_vel_y,ang_vel_z,lin_acc_x,lin_acc_y,lin_acc_z
0,0,4878.790299,-0.004261,-0.001065,-0.009587,-0.033519,-0.158017,9.797023
1,1,4878.792303,-0.007457,-0.005326,-0.005326,-0.052672,-0.134075,9.777869
2,2,4878.794301,-0.006392,-0.007457,-0.002131,-0.081402,-0.138863,9.715620
3,3,4878.796302,-0.005326,-0.007457,-0.005326,-0.076614,-0.143651,9.715620
4,4,4878.798302,-0.003196,-0.006392,-0.005326,-0.100556,-0.177170,9.658160
...,...,...,...,...,...,...,...,...
46048,46048,4970.886300,-0.002131,-0.001065,-0.004261,-0.014365,-0.244207,9.773081
46049,46049,4970.888300,-0.003196,-0.001065,-0.003196,-0.014365,-0.234631,9.792234
46050,46050,4970.890298,-0.002131,-0.001065,-0.003196,0.000000,-0.220265,9.777869
46051,46051,4970.892300,-0.005326,-0.001065,-0.005326,-0.043095,-0.220265,9.777869
